# Transfer learning

We will continue working on the chest x-ray dataset, from [Kermany et al. 2018](https://www.sciencedirect.com/science/article/pii/S0092867418301545?via%3Dihub)
 but this time we will stand closer to the original paper because we will use transfer learning.

![fig 1 of "Identifying Medical Diagnoses and Treatable Diseases by Image-Based Deep Learning" by Kermany et al.](images/tranfer_learning_xray.jpg)

The base model we will re-use is [ResNet50](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet50.html) from [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385).



With inspirations from [this github repo](https://github.com/liyu95/Deep_learning_examples/blob/master/4.ResNet_X-ray_classification/Densenet_fine_tune.ipynb) and [this kaggle thread](https://www.kaggle.com/code/iamsdt/transferlearning-pytorch-resnet-50)

In [ ]:
## on google colab, you will have to run the following line:
#!pip install pytorch-model-summary
#!wget https://github.com/Bjarten/early-stopping-pytorch/raw/refs/heads/main/early_stopping_pytorch/early_stopping.py
#!mv early_stopping.py pytorchtools.py

In [ ]:
## on google colab, you will have to run the following lines:

## downloading example images:
#!mkdir images
#!wget https://github.com/sib-swiss/pytorch-practical-training/raw/refs/heads/master/images/cat.jpg -O images/cat.jpg
#!wget https://github.com/sib-swiss/pytorch-practical-training/raw/refs/heads/master/images/car.jpg -O images/car.jpg
#!wget https://github.com/sib-swiss/pytorch-practical-training/raw/refs/heads/master/images/pandas_cat.jpg -O images/pandas_cat.jpg
#!wget https://github.com/sib-swiss/pytorch-practical-training/raw/refs/heads/master/images/NORMAL-1003233-0001.jpeg -O images/NORMAL-1003233-0001.jpeg
#!wget https://github.com/sib-swiss/pytorch-practical-training/raw/refs/heads/master/images/BACTERIA-1008087-0001.jpeg -O images/BACTERIA-1008087-0001.jpeg


## downloading an unzipping the full datasets (resized)
#!wget https://sibcloud-my.sharepoint.com/:u:/g/personal/wandrille_duchemin_sib_swiss/ESDXFrlw6JJGiK8X7xG4aVEB06cxW82KyK_KWXMwccIVhw?download=1 -O chest_xray_224.zip

#!mkdir data
#!unzip chest_xray_224.zip
#!mv chest_xray_224 data

## downloading embedded data
#!wget https://github.com/sib-swiss/pytorch-practical-training/raw/refs/heads/master/data/chest_Xray_train_embed.pt -O data/chest_Xray_train_embed.pt
#!wget https://github.com/sib-swiss/pytorch-practical-training/raw/refs/heads/master/data/chest_Xray_valid_embed.pt -O data/chest_Xray_valid_embed.pt
#!wget https://github.com/sib-swiss/pytorch-practical-training/raw/refs/heads/master/data/chest_Xray_train_y.pt -O data/chest_Xray_train_y.pt
#!wget https://github.com/sib-swiss/pytorch-practical-training/raw/refs/heads/master/data/chest_Xray_valid_y.pt -O data/chest_Xray_valid_y.pt



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import pytorch_model_summary as pms 

from torch.utils.data import TensorDataset, DataLoader

from torchvision.transforms import v2
import torchvision

from pytorchtools import EarlyStopping

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

# Best available weights (currently alias for IMAGENET1K_V2)
# Note that these weights may change across versions
RN50_model = resnet50(weights=ResNet50_Weights.DEFAULT).to(device)
RN50_model

The inference transforms are available at `ResNet50_Weights.IMAGENET1K_V2.transforms` 
and perform the following preprocessing operations: 
 * Accepts PIL.Image, batched (B, C, H, W) and single (C, H, W) image torch.Tensor objects. 
 * The images are resized to resize_size=[232] using interpolation=InterpolationMode.BILINEAR, 
 * followed by a central crop of crop_size=[224]. 
 * Finally the values are first rescaled to [0.0, 1.0] 
 * then normalized using mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].

In [ ]:
def make_np_img( img ):
    np_img = torch.moveaxis(img,0,2).numpy()    
    m = np_img.min()
    M = np_img.max()
    r = M-m
    return (np_img - m)/r
    

In [ ]:
from torchvision.io import read_image
RN50_transformer = ResNet50_Weights.IMAGENET1K_V2.transforms( antialias=True)

cat = read_image( "images/cat.jpg" )
cat_t = RN50_transformer(cat)

fig,ax = plt.subplots(1,2 , figsize = (10,5))
## Tensor are in C,H,W shape, but imshow wants H,W,C
## so we move axis 0 to index 2
ax[0].imshow( torch.moveaxis(cat,0,2).numpy())  
ax[1].imshow( make_np_img( cat_t ))

In [ ]:
RN50_dategories = np.array( ResNet50_Weights.DEFAULT.meta['categories'] )
RN50_dategories[:5]

In [ ]:
cat_t = cat_t.to(device)
with torch.no_grad():
    pred = RN50_model(cat_t.unsqueeze(0))
    class_probas = pred.squeeze(0).softmax(0).cpu()
    print( "class proba shape",class_probas.shape )
    print( 'probas:    ', class_probas[:5] , '...' )
    print( 'categories:',RN50_dategories[:5] , '...')
    


In [ ]:
def RN50_predict( img ):
    img_t = RN50_transformer( img )

    img_t = img_t.to(device)
    with torch.no_grad():
        pred = RN50_model(img_t.unsqueeze(0))
        class_probas = pred.squeeze(0).softmax(0)
    
    order = np.argsort( class_probas.cpu().numpy()  )[::-1]
    
    fig,ax = plt.subplots(1,3 , figsize = (10,5))
    ax[0].imshow(torch.moveaxis(img,0,2).cpu().numpy())
    ax[0].set_title(RN50_dategories[order[0]])
    ax[1].imshow( make_np_img( img_t.cpu() ) )
    ax[1].set_title(RN50_dategories[order[0]])
    
    sns.barplot( x = class_probas.cpu().numpy()[order[:5]], 
             y = RN50_dategories[order[:5]] , ax = ax[2])
    plt.tight_layout()
RN50_predict( cat )

In [ ]:
RN50_predict( read_image("images/car.jpg") )

In [ ]:
RN50_predict( read_image("images/pandas_cat.jpg") )

Now, if we try it on our x-ray images:

In [ ]:
## force the image to be RGB
img = read_image("images/NORMAL-1003233-0001.jpeg" , 
                 mode=torchvision.io.ImageReadMode.RGB )

RN50_predict(img)

In [ ]:
img = read_image("images/BACTERIA-1008087-0001.jpeg" , 
                 mode=torchvision.io.ImageReadMode.RGB )

RN50_predict(img)

As expected, it does not make sense.

---

## setting up transfer-learning

Rather than train a model from scratch as we did previously, we will leverage this complex model with [transfer learning](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-deep-learning-tips-and-tricks#parameter-tuning).

So we will freeze most layers of resnet50 and plug our own classifier on top.

![small Transfer Learning representation from https://stanford.edu/~shervine/teaching/cs-230/](https://stanford.edu/~shervine/teaching/cs-230/illustrations/transfer-learning-small-ltr.png?bee5e73de8fb2c6297a3a88804fabf5e)

In [ ]:
cat_t = RN50_transformer(cat).to(device)
print(pms.summary(RN50_model, cat_t.unsqueeze(0) , show_input=True))

In [ ]:
RN50_model

In [ ]:
# we freeze all layers by setting their parameters .requires_grad=False
for param in RN50_model.parameters():
    param.requires_grad = False

In [ ]:
## the layer we want to replace is called fc

# Parameters of newly constructed modules have requires_grad=True by default
input_dim = RN50_model.fc.in_features

RN50_model.fc = nn.Sequential(nn.Linear(input_dim , 1),
                              nn.Sigmoid() ).to(device)


In [ ]:
cat_t = RN50_transformer(cat).to(device)
print(pms.summary(RN50_model, cat_t.unsqueeze(0) , show_input=True))

## setup the dataloaders

In [ ]:
read_image_rgb = lambda x : read_image(x , mode=torchvision.io.ImageReadMode.RGB)

train_dataset = torchvision.datasets.ImageFolder('data/chest_xray_224/train', 
                                         loader = read_image_rgb,
                                         transform = RN50_transformer,
                                         target_transform = lambda x : torch.Tensor([x])  )
valid_dataset = torchvision.datasets.ImageFolder('data/chest_xray_224/test', 
                                         loader = read_image_rgb,
                                         transform = RN50_transformer,
                                         target_transform = lambda x : torch.Tensor([x])   )
    
    
batch_size = 64
train_dataloader = DataLoader( dataset= train_dataset , shuffle=True , batch_size = batch_size )
valid_dataloader = DataLoader( dataset= valid_dataset , shuffle=True , batch_size = batch_size )


In [ ]:
X,y = next(iter(train_dataloader))
X.shape , y.shape

In [ ]:
%%time
loss = nn.BCELoss()

with torch.no_grad():
    pred = RN50_model(X.to(device))
    print(pred.shape)
    print( 'avg loss:', loss( pred , y.to(device) ) )

From there we would train the model as we usually do.

However, in that particular case you can notice that just predicting 1 batch of 64 images took almost 4s on my measly CPU.

So to go through the 5,858 images of the dataset:

In [ ]:
((4/64)*5858)/60

I would need about 6 minutes.

So I would have fairly long epochs, and that is without even counting the backward pass.

Now, I could upgrade to a computer with a good GPU, but I could also apply reason like follow:

 * We have fixed all layers of Resnet50, expect the last
 * So at each epoch we compute the same thing, except for the last component
 * Going through all these layers is time consuming
 * Why not go though all these fixed layers once, and then train a model on this
 
Basically, we would use a truncated Resnet50 model as a way to embed our 224x224 image (~50,176 pixels) into a 2048 feature space.

And then we would train a classifier from that space

**exercise** : modify the last layer so it does not modify the output and compute the embedding for 1 batch.

In [ ]:
# %load solutions/TL_resnet50_embed.py

In [ ]:
with torch.no_grad():
    X,y = next(iter(train_dataloader))
    
    embedding = RN50_model(X.to(device))

print( embedding.shape )

In [ ]:
embedding

---
> What follow is the precomputation of the embeddings, which takes about 7min.

In [ ]:
# %load solutions/embed_all.py

Then we read the embedding from files where we have saved them:

In [ ]:

train_embedding = torch.load('data/chest_Xray_train_embed.pt')
train_Ys = torch.load('data/chest_Xray_train_y.pt')
valid_embedding = torch.load('data/chest_Xray_valid_embed.pt')
valid_Ys = torch.load('data/chest_Xray_valid_y.pt')

## Exercise: fit the embedded data

setup your dataloaders, create your model, and train it

 * a couple of linear + ReLu layers do the trick to learn from the embedding 
 * for the optimizer SGD with a learning rate of $10^{-2}$ worked in my tests 

setup dataloader

In [ ]:
# %load -r 1-17 solutions/TL_model_embed.py

define model

In [ ]:
# %load -r 18-44 solutions/TL_model_embed.py

define helper functions

In [ ]:
# %load -r 45-111 solutions/TL_model_embed.py

training setup

In [ ]:
# %load -r 112-127 solutions/TL_model_embed.py

initial round

In [ ]:
# %load -r 128-140 solutions/TL_model_embed.py

training

In [ ]:
# %load -r 141-171 solutions/TL_model_embed.py

results

In [ ]:
# %load -r 172- solutions/TL_model_embed.py



## merging the two models

In [ ]:
import copy
RN50_model.fc = copy.deepcopy( model.layers )

## saving and loading the model

In [ ]:
## saving 
torch.save(RN50_model, "modified_RN50.model") 

## loading
RN50_model_loaded = torch.load( "modified_RN50.model" , weights_only=False) 
RN50_model_loaded

## extra task: don't limit yourself to Neural Networks 

once you have the embedding you can apply whatever ML methodology you want.


In [ ]:
X_train = train_embedding.cpu().numpy()
y_train = train_Ys.cpu().squeeze().numpy()

In [ ]:
X_valid = valid_embedding.cpu().numpy()
y_valid = valid_Ys.cpu().squeeze().numpy()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
x_pca = pca.fit_transform( X_train )
pca.explained_variance_ratio_

In [ ]:
sns.scatterplot( x = x_pca[:,0] , y = x_pca[:,1] , hue = y_train )

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

logi_r=LogisticRegression(solver = 'liblinear',n_jobs=1) 

pipeline_lr=Pipeline([('model',logi_r)])



grid_values = {'model__C': np.logspace(-1,0,10),
               'model__penalty': ['l1'] }

grid_lr = GridSearchCV(pipeline_lr, 
                           param_grid = grid_values, 
                           scoring='accuracy',
                           cv=5,
                           n_jobs=1)
grid_lr.fit( X_train , y_train )
print('Grid best parameter (max. accuracy): ', grid_lr.best_params_)#get the best parameters
print('Grid best score (accuracy): ', grid_lr.best_score_)#get the best score calculated from the train/validation dataset
print('validation set score:' ,accuracy_score(y_valid , grid_lr.predict(X_valid)) )